<a href="https://colab.research.google.com/github/ysmnpksy/Final-Project/blob/main/Transformer-V1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1>Transformer Models</h1>


# Loading the Data

In [ ]:
! pip install datasets
! pip install transformers
! pip install datasets

In [ ]:
from google.colab import files

files.upload()

Saving kaggle.json to kaggle (1).json


{'kaggle.json': b'{"username":"yasminpaksoy","key":"9092d77ded0787db0dc92dec0c6c058c"}'}

In [ ]:
# making kaggle directory 
! mkdir ~/.kaggle

# copying api login info into directory 
! cp kaggle.json ~/.kaggle/

# allocating required permissions 
! chmod 600 ~/.kaggle/kaggle.json

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [ ]:
! kaggle datasets download rmisra/imdb-spoiler-dataset

# unzipping dataset 
! unzip imdb-spoiler-dataset.zip

imdb-spoiler-dataset.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  imdb-spoiler-dataset.zip
replace IMDB_movie_details.json? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace IMDB_reviews.json? [y]es, [n]o, [A]ll, [N]one, [r]ename: n


In [ ]:
# importing pandas to read the JSON files
import pandas as pd 

# information regarding reviews file
all_reviews = pd.read_json('../content/IMDB_reviews.json', lines=True)

print('Total number of reviews:', all_reviews['review_date'].count())
print('Total number of reviews that contain spoilers:', all_reviews['is_spoiler'].sum())
print('User reviews shape:', all_reviews.shape)
print()

print('First 5 user reviews:')
all_reviews.head()

Total number of reviews: 573913
Total number of reviews that contain spoilers: 150924
User reviews shape: (573913, 7)

First 5 user reviews:


,review_date,movie_id,user_id,is_spoiler,review_text,rating,review_summary
0,10 February 2006,tt0111161,ur1898687,True,"In its Oscar year, Shawshank Redemption (writt...",10,A classic piece of unforgettable film-making.
1,6 September 2000,tt0111161,ur0842118,True,The Shawshank Redemption is without a doubt on...,10,Simply amazing. The best film of the 90's.
2,3 August 2001,tt0111161,ur1285640,True,I believe that this film is the best story eve...,8,The best story ever told on film
3,1 September 2002,tt0111161,ur1003471,True,"**Yes, there are SPOILERS here**This film has ...",10,Busy dying or busy living?
4,20 May 2004,tt0111161,ur0226855,True,At the heart of this extraordinary movie is a ...,8,"Great story, wondrously told and acted"


# Preparing the Data <a class="anchor" id="preparing"></a>

In [ ]:
# Loading in data from JSON format to lists 
# Only loading review text and is_spoiler field

import json

dataset = [] 

# loop appending each review and label to list 
with open('IMDB_reviews.json', 'r') as json_file:
  for jsonObj in json_file:
    data = json.loads(jsonObj)
    if data['is_spoiler'] == True:
      dataset.append([1, data['review_text']])
    else:
      dataset.append([0, data['review_text']])

In [ ]:
# suffling data
import numpy as np
np.random.shuffle(dataset)

# setting number of training and testing samples
training_samples = 523913
testing_samples = 50000

# splitting into training and testing sets
train = dataset[:training_samples]
test = dataset[training_samples: training_samples + testing_samples]

In [ ]:
print('Number of training samples:',len(train))
print('Number of testing samples:',len(test))

Number of training samples: 523913
Number of testing samples: 50000


Here I am further splitting the training set into partial training and validation. This ensures that the data used to validate will not be the same as that used to train, helping avoid overfitting issues. I will be training on 151,848 samples and validating on 50,000. 

In [ ]:
# setting number of training and validation samples
partial_training_samples = 3913
validation_samples = 50000

# splitting into training and validation sets
partial_train_unbal = dataset[:partial_training_samples]
val = dataset[partial_training_samples: partial_training_samples + validation_samples]

In [ ]:
print('Number of partial training samplees:',len(partial_train_unbal))
print('Number of validation sampels:',len(val))

Number of partial training samplees: 3913
Number of validation sampels: 50000


In [ ]:
spoilers = 0;
nonSpoilers = 0;

for i in partial_train_unbal: 
  if i[0] == 1: 
    spoilers += 1 
  else: 
    nonSpoilers += 1

print("Total number of reviews: " + str(len(partial_train_unbal)))
print("Total nummber with spoilers: " + str(spoilers))
print("Total number without spoilers " + str(nonSpoilers))

Total number of reviews: 3913
Total nummber with spoilers: 983
Total number without spoilers 2930


In [ ]:
partial_train = [] 
sum = 0

for data in range(len(partial_train_unbal)):
  if partial_train_unbal[data][0] == 1:
    partial_train.append([1, partial_train_unbal[data][1]])
  else:
    if sum < spoilers:
      sum += 1
      partial_train.append([0, partial_train_unbal[data][1]])

In [ ]:
spoilers = 0;
nonSpoilers = 0;

for i in partial_train: 
  if i[0] == 1: 
    spoilers += 1 
  else: 
    nonSpoilers += 1

print("Total nummber with spoilers: " + str(spoilers))
print("Total number without spoilers " + str(nonSpoilers))

Total nummber with spoilers: 983
Total number without spoilers 983


In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

In [ ]:
def preprocess_function(examples):
    return tokenizer(examples['1'], truncation=True)

In [ ]:
from datasets import Dataset
train = Dataset.from_pandas(pd.DataFrame(train))
partial_train = Dataset.from_pandas(pd.DataFrame(partial_train))
val = Dataset.from_pandas(pd.DataFrame(val))
test = Dataset.from_pandas(pd.DataFrame(test))

In [ ]:
tokenized_partial_train = partial_train.map(preprocess_function, batched=True)

  0%|          | 0/2 [00:00<?, ?ba/s]

In [ ]:
pre_tokenizer_columns = set(partial_train.features)
tokenizer_columns = list(set(tokenized_partial_train.features) - pre_tokenizer_columns)
print("Columns added by tokenizer:", tokenizer_columns)
print("All columns:", list(set(tokenized_partial_train.features)))

Columns added by tokenizer: ['attention_mask', 'input_ids']
All columns: ['input_ids', '0', 'attention_mask', '1']


In [ ]:
tokenized_val = val.map(preprocess_function, batched=True)

  0%|          | 0/50 [00:00<?, ?ba/s]

In [ ]:
tokenized_test = test.map(preprocess_function, batched=True)

  0%|          | 0/50 [00:00<?, ?ba/s]

In [ ]:
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="tf")

In [ ]:
tf_partial_train = tokenized_partial_train.to_tf_dataset(
    columns=tokenizer_columns,
    label_cols=["0"],
    shuffle=False,
    batch_size=16,
    collate_fn=data_collator
)

tf_val = tokenized_val.to_tf_dataset(
    columns=tokenizer_columns,
    label_cols=["0"],
    shuffle=False,
    batch_size=16,
    collate_fn=data_collator,
)

In [ ]:
tf_test = tokenized_test.to_tf_dataset(
    columns=tokenizer_columns,
    label_cols=["0"],
    shuffle=False,
    batch_size=16,
    collate_fn=data_collator,
)

# Build Model

In [ ]:
from transformers import create_optimizer
import tensorflow as tf

batch_size = 16
num_epochs = 5
batches_per_epoch = len(tokenized_partial_train) // batch_size
total_train_steps = int(batches_per_epoch * num_epochs)
optimizer, schedule = create_optimizer(init_lr=2e-5, num_warmup_steps=0, num_train_steps=total_train_steps)

In [ ]:
from transformers import TFAutoModelForSequenceClassification
model = TFAutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertForSequenceClassification: ['vocab_transform', 'vocab_layer_norm', 'activation_13', 'vocab_projector']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier', 'pre_classifier', 'dropout_19']
You should probably TRAIN this model on a down-stream task to be able to use i

In [ ]:
import tensorflow as tf
model.compile(optimizer=optimizer)

No loss specified in compile() - the model's internal loss computation will be used as the loss. Don't panic - this is a common way to train TensorFlow models in Transformers! To disable this behaviour, please pass a loss argument, or explicitly pass `loss=None` if you do not want your model to compute a loss.


In [ ]:
model.fit(x=tf_partial_train, validation_data=tf_val, epochs=3)

Epoch 1/3
123/123 [==============================] - 15355s 126s/step - loss: 0.4771 - val_loss: 2.9292
Epoch 2/3
123/123 [==============================] - 15401s 126s/step - loss: 0.6251 - val_loss: 2.4067
Epoch 3/3
123/123 [==============================] - 15472s 127s/step - loss: 0.6841 - val_loss: 1.6056


In [ ]:
from transformers import Trainer
y_pred = model.predict(tf_test)

In [ ]:
pred_labels = np.argmax(y_pred.logits, axis=1)

In [ ]:
pred_labels

array([1, 1, 1, ..., 1, 1, 1])

In [ ]:
print(y_pred.loss)

None


In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, make_scorer
print('Accuracy: %.2f' % (accuracy_score(tokenized_test['0'], pred_labels)))
print('Precision score: %.2f' % (precision_score(tokenized_test['0'], pred_labels)))
print('Recall score: %.2f' % (recall_score(tokenized_test['0'], pred_labels)))
print('F1 score: %.2f' % (f1_score(tokenized_test['0'], pred_labels)))

Accuracy: 0.26
Precision score: 0.26
Recall score: 1.00
F1 score: 0.42


In [ ]:
y_pred

TFSequenceClassifierOutput([('logits', array([[-0.99175286,  1.0261542 ],
                                    [-0.98673147,  1.0146419 ],
                                    [-0.99399686,  1.0370796 ],
                                    ...,
                                    [-0.99542165,  1.0357063 ],
                                    [-0.99485433,  1.0360426 ],
                                    [-0.9882538 ,  1.0131129 ]], dtype=float32))])

# Save Model

In [ ]:
import pickle
with open('model.pickle', 'wb') as f:
  pickle.dump(model, f)

INFO:tensorflow:Assets written to: ram://93a0c5ec-b1f1-4b2d-b23a-1ec66f8309f7/assets


INFO:tensorflow:Assets written to: ram://93a0c5ec-b1f1-4b2d-b23a-1ec66f8309f7/assets


In [ ]:
import os
os.listdir()

['.config',
 'IMDB_reviews.json',
 'kaggle (1).json',
 'kaggle.json',
 'model.pickle',
 'IMDB_movie_details.json',
 'imdb-spoiler-dataset.zip',
 'sample_data']

In [ ]:
with open('model.pickle', 'rb') as f:
  model = pickle.load(f)
type(model)